<a href="https://colab.research.google.com/github/dudwn98/iipl_lda_topic_modeling/blob/main/LDA_TopicModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import argparse
import pandas as pd
import re
import konlpy
import argparse

In [111]:
def clean_doc(news):
    okt = konlpy.tag.Okt()
    clean_tokens = []

    #개행 제거
    news = re.sub('\\\\n', '', news)
    #이메일 제거
    news = re.sub(r'\b[\w\+]+@[\w]+.[\w]+.[\w]+.[\w]+\b', ' ', news)
    #특수문자, 자음 제거
    news = re.sub(r'[^ㄱ-ㅣ가-힣A-Za-z0-9]',' ', news)
    #중복 공백 제거
    news = re.sub(' +', ' ', news)

    return news

In [115]:
def remove_stopwords(text):
    okt = konlpy.tag.Okt()
    
    tokens = okt.nouns(text)

    stops = ['기자', '뉴스', '경향', '뉴시스', '무단 배포', '금지', '무단',
             '위해', '보기', '배포', '비롯', '통해', '통한', '마련', '다른',
             '가지', '여개', '이데일리']
    meaningful_words = [w for w in tokens if not w in stops and len(w)>1]
    return ' '.join(meaningful_words)

In [116]:
path='/content/drive/MyDrive/크롤링 데이터/2018/2018백남준기념관.csv'

file = pd.read_csv(path, encoding='utf-8')
df = pd.DataFrame({'doc' : file.기사제목 + file.기사내용})

df.loc[:, 'doc'] = df['doc'].apply(preprocessing)
df.loc[:, 'doc'] = df['doc'].apply(remove_stopwords)
print(df)

## 텍스트 데이터를 단어 빈도수에 기반해 벡터화시키기
# max_df = 토큰이 나타날 최대 문서 개수 -> 소수점이면 비중
# max_features = corpus 중 빈도수가 가장 높은 순으로 해당 개수만큼만 뽑아냄
# min_df = 토큰이 나타날 최소 문서 개수 -> 정수면 횟수
# ngram_range = (min_n, max_n) 튜플, 단어장 생성에 사용할 토큰의 크기를 결정
count_vect = CountVectorizer(max_df=0.95, max_features=1000,
                            min_df=2, ngram_range=(1,1))
# feature 리스케일링 해서 feature 평균이 0 분산이 1 되게 만들어줌
#fit_transfrom은 train dataset에서만 사용
ftr_vect = count_vect.fit_transform(df.doc)

# LDA 클래스 사용해 피쳐 벡터화시킨 거 토픽모델링
# n_components = 토픽개수
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(ftr_vect)

# 각 토픽
print(lda.components_.shape)
# 단어들을 벡터화한 feature
print(lda.components_)

                                                  doc
0   백남준 마지막 식사 장어 태창 방직 대표 부친 재산 국고 몰수 뉴욕 악전 고투 전인...
1   서울 과거 현재 미래 팔색조 창신동 으리 건물 인테리어 여행 사람 냄새 폴폴 가파른...
2   추석 서울 여행 힙스터 골목길 투어 성수동 나비 정원 이하 한국 관광 공사 제공 서...
3   서민 전통 시장 여행 향기 서울 관광 재단 서울 이야기 재미 시장 이야기 전통 시장...
4   천재 화가 이인성 고택 근대 문화 전시관 복원 콘텐츠 고택 형태 결정 우선 진품 사...
5   정보 골목 간직 창신동 앵커 정보 시간 미세먼지 때문 고생 요즘 날씨 걷기 봄날 미...
6   대한민국 공간 화대 행복 마을 숭인 품격 생활 공간 수여 대한민국 공간 문화 대상 ...
7   도시 문재인 뉴딜 예산 집행 도시 재생 사업 추진 올해 예산 지역 오후 인천 남동구...
8   인터뷰 병용 서울시 주거 사업 기획 도시재생 사업 도시재생 사업 시작 서울시 주거지...
9   국립 싱가포르 교수 학생 서울 도시재생 서울 박대 싱가포르 국립대 디자인 환경 대학...
10  투기 도시재생 사업 국토교통부 서울 창신 숭인 지구 도시재생 선도 지역 선정 마을 ...
11  서울시 민선 박물관 미술관 생활사 박물관 사진 미술관 로봇 과학관 공간 조성 현재 ...
12  도시재생 사업 투기 세력 창신 숭인 지구 기획 부동산 활개 세운상가 임대료 대폭 마...
13  서울시 창신 숭인 채석장 도시재생 암초 토지 확보 사업 지연 사업 부지 차지 경찰 ...
14  서울 시립 미술관 분관 오픈 시민 가까이 서울 예고 인근 평창 내달 착공 사진 미술...
15  박원순 도시 계획 낙후 지역 도시재생 현장 목소리 선거 도루묵 개발 도시재생 외면 ...
16  대표 소장품 시립 미술관 명화 산책 개관 주년 기념 세대 불화 이성자 무제 부터 천...
17  정부 주도 도시재생 뉴딜 로드맵 한계 우려 

In [117]:
# lda_model = 벡터화시킨 텍스트 데이터를 fit까지만 적용한 모델
def display_topic_words(lda_model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(lda_model.components_):
        print('\nTopic #', topic_idx+1)

        # topic 별로 1000개의 features 중에서 높은 값 순으로 정렬 후 index 반환
        #argsort = 디폴트가 오름차순 -> [::-1]로 내림차순으로 변경
        topic_word_idx = topic.argsort()[::-1]
        top_idx = topic_word_idx[:num_top_words]

        # CountVectorizer 함수 할당시킨 객체에 get_feature_names()로 벡터화시킨 features
        # 벡터화시킨 feature는 숫자-알파벳 순으로 정렬, 단어들 순서는 fit_transform 시키고 난 이후에도 동일
        feature_concat = '+'.join([str(feature_names[i])+'*'+str(round(topic[i], 1)) for i in top_idx])
        print(feature_concat)

feature_names = count_vect.get_feature_names()
display_topic_words(lda, feature_names, 5)


Topic # 1
창신동*55.0+시장*54.2+골목*52.2+봉제*39.2+거리*23.2

Topic # 2
미술관*112.0+시립*63.2+본관*31.2+시설*26.3+박물관*24.0

Topic # 3
사업*94.5+도시재생*56.0+주민*46.2+지역*44.0+숭인*28.4

Topic # 4
사업*91.7+도시재생*87.1+주민*44.2+임대료*29.2+숭인*28.0

Topic # 5
고용*147.2+효과*137.2+도시재생*86.6+유발*77.2+일자리*66.2


In [93]:
doc_topics = lda.transform(ftr_vect)
print(doc_topics.shape)
print(doc_topics)

(8, 5)
[[9.97171827e-01 7.04595053e-04 7.09844429e-04 7.09138450e-04
  7.04595053e-04]
 [9.73971360e-04 9.66613309e-04 7.15351969e-01 2.81740833e-01
  9.66613309e-04]
 [9.97720806e-01 5.66945336e-04 5.71807629e-04 5.73496030e-04
  5.66945336e-04]
 [1.56979397e-01 1.47142293e-03 1.48502349e-03 8.38592734e-01
  1.47142293e-03]
 [8.64321432e-04 8.58599150e-04 9.96546804e-01 8.71675842e-04
  8.58599150e-04]
 [9.98536530e-01 3.63866082e-04 3.68220845e-04 3.67516580e-04
  3.63866082e-04]
 [9.94725335e-01 1.31634070e-03 1.32090086e-03 1.32108278e-03
  1.31634070e-03]
 [9.97399431e-01 6.47652158e-04 6.52894122e-04 6.52370986e-04
  6.47652158e-04]]


In [67]:
topic_names = ['Topic #' + str(i) for i in range(0,5)]
topic_df = pd.DataFrame(data=doc_topics, columns=topic_names)
print(topic_df)

   Topic #0  Topic #1  Topic #2  Topic #3  Topic #4
0  0.998493  0.000377  0.000375  0.000378  0.000377
1  0.000655  0.000652  0.000652  0.997378  0.000664
2  0.998888  0.000278  0.000277  0.000279  0.000278
3  0.001248  0.001248  0.001242  0.001252  0.995010
4  0.000619  0.142411  0.000614  0.855737  0.000620
5  0.999234  0.000192  0.000190  0.000192  0.000192
6  0.997309  0.000674  0.000671  0.000673  0.000673
7  0.998617  0.000345  0.000344  0.000346  0.000347
